In [1]:
%cd /home/jkliu/Projects/CSAServer

/home/jkliu/Projects/CSAServer


/home/jkliu/Projects/CSAServer/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from fastapi.testclient import TestClient
from main import app

client = TestClient(app)

cg_container: INFO     Load cg for com.socialnmobile.dictapps.notepad.color.note-15600
cg_container: INFO     Load cg for com.amaze.filemanager-117
cg_container: INFO     Load cg for com.imdb.mobile-109070200
cg_container: INFO     Load cg for com.shaiban.audioplayer.mplayer-1007102000
cg_container: INFO     Load cg for org.woheller69.spritpreise-24
cg_container: INFO     Load cg for org.wikipedia.alpha-50476
cg_container: INFO     Load cg for video.downloader.videodownloader-128
cg_container: INFO     Load cg for com.adsk.sketchbook-454083
cg_container: INFO     Load cg for org.mozilla.firefox-2016048879
cg_container: INFO     Load cg for com.dictionary-629
cg_container: INFO     Load cg for org.wikipedia.dev-50496
cg_container: INFO     Load cg for com.zhiliaoapp.musically-2022903010
cg_container: INFO     Load cg for com.accuweather.android-200300002
cg_container: INFO     Load cg for todolist.scheduleplanner.dailyplanner.todo.reminders-1000244
cg_container: INFO     Load cg for org

In [4]:
# test get_acitivities
request = client.build_request(
    method="post",
    url="get_permission?package_name=org.wikipedia.dev&version_code=50496"
)
response = client.send(request)
result = response.json()
print(result["permissions"])
print(result["last_debloated_permissions"])


httpx       : INFO     HTTP Request: POST http://testserver/get_permission?package_name=org.wikipedia.dev&version_code=50496 "HTTP/1.1 200 OK"


['android.permission.FOREGROUND_SERVICE', 'android.permission.POST_NOTIFICATIONS', 'android.permission.WAKE_LOCK', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_NETWORK_STATE']
[]


In [5]:
# test get_acitivities
request = client.build_request(
    method="post",
    url="get_permission?package_name=com.shaiban.audioplayer.mplayer&version_code=1007102000"
)
response = client.send(request)
result = response.json()
print(result["permissions"])
print(result["last_debloated_permissions"])


httpx       : INFO     HTTP Request: POST http://testserver/get_permission?package_name=com.shaiban.audioplayer.mplayer&version_code=1007102000 "HTTP/1.1 200 OK"


['android.permission.REQUEST_IGNORE_BATTERY_OPTIMIZATIONS', 'android.permission.FOREGROUND_SERVICE', 'android.permission.BLUETOOTH_ADVERTISE', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.BLUETOOTH', 'android.permission.CHANGE_WIFI_STATE', 'android.permission.READ_MEDIA_AUDIO', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.BLUETOOTH_ADMIN', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.ACCESS_WIFI_STATE', 'android.permission.INTERNET', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.NEARBY_WIFI_DEVICES', 'android.permission.SCHEDULE_EXACT_ALARM', 'android.permission.BLUETOOTH_CONNECT', 'android.permission.READ_MEDIA_VIDEO', 'android.permission.WRITE_SETTINGS', 'android.permission.CHANGE_NETWORK_STATE', 'android.permission.BLUETOOTH_SCAN', 'android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.WAKE_LOCK', 'android.permission.READ_MEDIA_IMAGES', 'android.permission.WRITE_EXTERNAL_STORAGE']
[]


In [8]:
import os
# test get_acitivities
app_name = "com.shaiban.audioplayer.mplayer"
version_code = "1007102000"
result_dir = "tests/results/temp_debloated"

if os.path.exists(result_dir):
    os.system(f"rm -r {result_dir}")
os.makedirs(result_dir)

from datetime import datetime
import os
request = client.build_request(
    method="post",
    url=f"debloat_permission?package_name={app_name}&version_code={version_code}&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION"
)
response = client.send(request)
result = response.json()
to_remove_methods = result["to_remove_methods"]
print(to_remove_methods)
print(result["to_remove_permissions"])



Main        : INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION for com.shaiban.audioplayer.mplayer-1007102000
MethodFinder: INFO     Debloat permission ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION'] for com.shaiban.audioplayer.mplayer-1007102000
httpx       : INFO     HTTP Request: POST http://testserver/debloat_permission?package_name=com.shaiban.audioplayer.mplayer&version_code=1007102000&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION "HTTP/1.1 200 OK"


['<com.inmobi.media.t6: java.util.HashMap c()>', '<com.inmobi.media.t6: boolean d()>', '<androidx.appcompat.app.i0: android.location.Location b()>', '<com.inmobi.media.ve: boolean a()>', '<com.inmobi.media.kc: void d()>', '<com.applovin.impl.sdk.s: boolean f()>', '<com.inmobi.media.t6: java.util.HashMap a(android.location.Location,boolean,android.location.Location)>', '<com.inmobi.sdk.InMobiSdk: void a(android.content.Context,com.inmobi.sdk.SdkInitializationListener,java.lang.String,org.json.JSONObject)>', '<com.google.android.gms.nearby.connection.ConnectionsStatusCodes: java.lang.String getStatusCodeString(int)>', '<com.applovin.impl.sdk.s: boolean c()>', '<com.google.android.gms.internal.ads.zzcgl: void onGeolocationPermissionsShowPrompt(java.lang.String,android.webkit.GeolocationPermissions$Callback)>', '<bo.o: void <clinit>()>', '<com.inmobi.sdk.InMobiSdk: void b()>', '<com.inmobi.media.d2: boolean b()>']
['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARS

In [4]:
import os


def sign_align_new_apk(input_apk):
    aligned_apk = input_apk.replace(".apk", "_aligned_tmp.apk")
    output_apk = input_apk.replace(".apk", "_signed.apk")
    temp_file = input_apk.replace(".apk", "_signed.apk.idsig")
    os.system(f"zipalign -f 4 {input_apk} {aligned_apk}")
    os.system(f"apksigner sign --ks Apksigner --ks-pass pass:60948491 --out {output_apk} {aligned_apk}")
    # os.system(f"zip -d {output_apk} META-INF/CERT.SF META-INF/CERT.RSA")
    # os.system(f"mv {output_apk} {input_apk}")
    # os.system(f"rm {aligned_apk}")
    # os.system(f"rm {temp_file}")

In [6]:
# test get_acitivities
app_name = "org.wikipedia.dev"
version_code = "50496"
result_dir = "tests/results/temp_debloated"

if os.path.exists(result_dir):
    os.system(f"rm -r {result_dir}")
os.makedirs(result_dir)

from datetime import datetime
import os
request = client.build_request(
    method="post",
    url=f"debloat_permission?package_name={app_name}&version_code={version_code}&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION"
)
response = client.send(request)
result = response.json()
to_remove_methods = result["to_remove_methods"]
print(to_remove_methods)
print(result["to_remove_permissions"])

current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
current_time = current_time.replace(" ", "_")
with open(f"{result_dir}/{current_time}_to_remove.log", "w") as write2filelog:
    for method in to_remove_methods:
        write2filelog.write(f"{method}\n")

cmd = f"java -jar lib/debloater/target/debloater-1.0-SNAPSHOT-jar-with-dependencies.jar -toDebloatPath {result_dir}/{current_time}_to_remove.log -androidJar lib/platforms -apkPath apks/{app_name}-{version_code}.apk"
os.system(cmd)
sign_align_new_apk(f"{result_dir}/{app_name}-{version_code}.apk")


Main        : INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION for org.wikipedia.dev-50496
MethodFinder: INFO     Debloat permission ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION'] for org.wikipedia.dev-50496
httpx       : INFO     HTTP Request: POST http://testserver/debloat_permission?package_name=org.wikipedia.dev&version_code=50496&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION "HTTP/1.1 200 OK"


['<org.maplibre.android.location.permissions.PermissionsManager: boolean isFineLocationPermissionGranted(android.content.Context)>', '<org.wikipedia.places.PlacesFragment: boolean haveLocationPermissions()>', '<org.maplibre.android.location.permissions.PermissionsManager: boolean isCoarseLocationPermissionGranted(android.content.Context)>', '<org.wikipedia.places.PlacesFragment: void locationPermissionRequest$lambda$2(org.wikipedia.places.PlacesFragment,java.util.Map)>', '<org.wikipedia.places.PlacesFragment: void onCreateView$lambda$13(org.wikipedia.places.PlacesFragment,android.view.View)>', '<androidx.appcompat.app.TwilightManager: android.location.Location getLastKnownLocation()>', '<org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity,boolean,boolean)>', '<org.wikipedia.places.PlacesFragment: void onViewCreated$lambda$25$lambda$24(org.wikipedia.places.PlacesFragment,org.maplibre.android.maps.MapLibreMap,org.maplibre.and

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Initialized soot
Number of to debloate methods: 9
/home/jkliu/Projects/CSAServer/tests/results/temp_debloatedapks/org.wikipedia.dev-50496.apk
Removed [<org.wikipedia.places.PlacesFragment: void locationPermissionRequest$lambda$2(org.wikipedia.places.PlacesFragment,java.util.Map)>, <org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity,boolean,boolean)>, <org.wikipedia.places.PlacesFragment: void onViewCreated$lambda$25$lambda$24(org.wikipedia.places.PlacesFragment,org.maplibre.android.maps.MapLibreMap,org.maplibre.android.maps.Style)>, <org.maplibre.android.location.permissions.PermissionsManager: boolean isFineLocationPermissionGranted(android.content.Context)>, <org.maplibre.android.location.permissions.PermissionsManager: boolean isCoarseLocationPermissionGranted(android.content.Context)>, <androidx.appcompat.app.TwilightManager: android.location.Location getLastKnownLocation()>, <org.wikipedia.places.PlacesFragment: void 

In [3]:
# test get_acitivities
app_name = "org.wikipedia.dev"
version_code = "50496"
result_dir = "tests/results/temp_debloated"

import json
import os

if os.path.exists(result_dir):
    os.system(f"rm -r {result_dir}")
os.makedirs(result_dir)

permission_dict = {}
permission_dict["permissions"] = "android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION"

permission_json = json.dumps(permission_dict)

from datetime import datetime
import os
request = client.build_request(
    method="post",
    # url=f"debloat_permission?package_name={app_name}&version_code={version_code}",
    url=f"debloat_permission_test?package_name={app_name}&version_code={version_code}",
    # url=f"debloat_permission_old?package_name={app_name}&version_code={version_code}&permissions=android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION",
    content=permission_json
)
response = client.send(request)
result = response.json()
print(result)


# to_remove_methods = result["to_remove_methods"]
# print(to_remove_methods)
# print(result["to_remove_permissions"])

# current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# current_time = current_time.replace(" ", "_")
# with open(f"{result_dir}/{current_time}_to_remove.log", "w") as write2filelog:
#     for method in to_remove_methods:
#         write2filelog.write(f"{method}\n")

# cmd = f"java -jar lib/debloater/target/debloater-1.0-SNAPSHOT-jar-with-dependencies.jar -toDebloatPath {result_dir}/{current_time}_to_remove.log -androidJar lib/platforms -apkPath apks/{app_name}-{version_code}.apk"
# os.system(cmd)
# print("*******************FINISH********************")
# sign_align_new_apk(f"{result_dir}/{app_name}-{version_code}.apk")


Main        : INFO     Debloat permission android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_COARSE_LOCATION for org.wikipedia.dev-50496
MethodFinder: INFO     Debloat permission ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION'] for org.wikipedia.dev-50496
httpx       : INFO     HTTP Request: POST http://testserver/debloat_permission_test?package_name=org.wikipedia.dev&version_code=50496 "HTTP/1.1 200 OK"


{'to_remove_methods': ['<org.wikipedia.places.PlacesFragment: boolean haveLocationPermissions()>', '<org.wikipedia.places.PlacesFragment: void onViewCreated$lambda$25$lambda$24(org.wikipedia.places.PlacesFragment,org.maplibre.android.maps.MapLibreMap,org.maplibre.android.maps.Style)>', '<org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity)>', '<org.maplibre.android.location.permissions.PermissionsManager: boolean isCoarseLocationPermissionGranted(android.content.Context)>', '<org.maplibre.android.location.permissions.PermissionsManager: void requestLocationPermissions(android.app.Activity,boolean,boolean)>', '<org.wikipedia.places.PlacesFragment: void onCreateView$lambda$13(org.wikipedia.places.PlacesFragment,android.view.View)>', '<org.wikipedia.places.PlacesFragment: void locationPermissionRequest$lambda$2(org.wikipedia.places.PlacesFragment,java.util.Map)>', '<androidx.appcompat.app.TwilightManager: android.location.Loca